In [1]:
import sys
sys.path.append("../../")

from src.sslHiC import get_DCIs
from scipy.io import mmread
from scipy import sparse
from sklearn import metrics

/data/lihan/miniconda/envs/sslHiC-test/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
resolution = '50kb'
folds = [2,4,6]

In [3]:
for fold in folds:
    c1 = mmread(f'dataset/GM12878_combined_chr21_{fold}fold_c1.mtx')
    c2 = mmread(f'dataset/GM12878_combined_chr21_{fold}fold_c2.mtx')
    c1 = sparse.triu(c1,1) + sparse.triu(c1).transpose()
    c2 = sparse.triu(c2,1) + sparse.triu(c2).transpose()
    label_mat = mmread(f'dataset//GM12878_combined_chr21_{fold}fold_label.mtx')
    label_mat = sparse.triu(label_mat,1) + sparse.triu(label_mat).transpose()
    labels = label_mat.toarray().reshape(-1)

    preds = get_DCIs(m1=c1, m2=c2, resol=resolution, filter_zero_interactions=True)

    fpr, tpr, thresholds = metrics.roc_curve(labels, preds.reshape(-1), pos_label=1)
    lr_precision, lr_recall, thresholds =metrics.precision_recall_curve(labels, preds.reshape(-1),pos_label=1)
    print(f"[{fold}fold]  AUROC:", metrics.auc(fpr, tpr), "AUPRC:", metrics.auc(lr_recall, lr_precision))

[2fold]  AUROC: 0.9287950202691221 AUPRC: 0.8196444177210571
[4fold]  AUROC: 0.9827068841425813 AUPRC: 0.9576800463352986
[6fold]  AUROC: 0.9890200305762352 AUPRC: 0.9731552260313818
